<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/05-LangChain_Redis/05.04_AWS_Bedrock_LangChain_Redis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Question Answering with LangChain, Amazon Bedrock and Redis

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

This notebook would use Amazon Bedrock, Redis with Vector Similarity Search and LangChain to answer questions about the information contained in a document.

### Install Dependencies


In [28]:
!pip install -q redis  langchain tiktoken boto3 unstructured


## Initialize AWS Client

Enter your AWS credentials here

In [29]:
aws_access_key_id="AKIAY3LZERKGEXAMPLE" # @param {type:"string"}
aws_secret_access_key='XXXXXX' # @param {type:"string"}
region_name='us-west-2' # @param {type:"string"}

In [30]:
# read secure key with getpass() in case of screen share to protect the key
from getpass import getpass
if aws_secret_access_key == 'XXXXXX':
  aws_secret_access_key = getpass("AWS Secret Access Key:")

AWS Secret Access Key:··········


In [31]:
import boto3
bedrock = boto3.client(service_name='bedrock',aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key,region_name=region_name)
bedrock_runtime = boto3.client(service_name='bedrock-runtime',aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key,region_name=region_name)

### List Bedrock foundational models

In [32]:
#list foundational models (optional)
model_list=bedrock.list_foundation_models()
from pprint import pprint
#pprint(model_list)
for model in model_list["modelSummaries"]:
  print(model["outputModalities"], " ", model["modelId"])

['TEXT']   amazon.titan-tg1-large
['EMBEDDING']   amazon.titan-e1t-medium
['EMBEDDING']   amazon.titan-embed-g1-text-02
['TEXT']   amazon.titan-text-express-v1
['EMBEDDING']   amazon.titan-embed-text-v1
['IMAGE']   stability.stable-diffusion-xl
['IMAGE']   stability.stable-diffusion-xl-v0
['TEXT']   ai21.j2-grande-instruct
['TEXT']   ai21.j2-jumbo-instruct
['TEXT']   ai21.j2-mid
['TEXT']   ai21.j2-mid-v1
['TEXT']   ai21.j2-ultra
['TEXT']   ai21.j2-ultra-v1
['TEXT']   anthropic.claude-instant-v1
['TEXT']   anthropic.claude-v1
['TEXT']   anthropic.claude-v2
['TEXT']   cohere.command-text-v14


### Install Redis Stack

Redis Search will be used as Vector Similarity Search engine for LangChain. Instead of using in-notebook Redis Stack https://redis.io/docs/getting-started/install-stack/ you can provision your own free instance of Redis in the cloud. Get your own Free Redis Cloud instance at https://redis.com/try-free/

In [33]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failed writing body


### Connect to Redis

By default this notebook would connect to the local instance of Redis Stack. If you have your own Redis Cloud instance - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [34]:
import redis
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

#shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test Redis connection
!redis-cli $REDIS_CONN PING

In [35]:
from langchain.llms import Bedrock
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores.redis import Redis
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains import RetrievalQA

### Load text and split it into managable chunks

Without this step any large body of text would exceed the limit of tokens you can feed to the LLM

In [36]:
# Add your own URLs here
urls = [
    "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"
]
loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index = True)
texts = text_splitter.split_documents(documents)
# Optionally examine the result of text load+splitting
#for text in texts:
#  print(text)

### Initialize embeddings engine

Using AWS Bedrock Embeddings API.

In [37]:
embeddings = BedrockEmbeddings(client=bedrock_runtime)


### Initialize LLM

In this notebook we are using Amazon Bedrock LLM.

Try to replace `amazon.titan-tg1-large` with one of the models from the list above.

In [38]:
llm = Bedrock(model_id="amazon.titan-tg1-large",client=bedrock_runtime)


### Create vector store from the documents using Redis as Vector Database

In [39]:
def get_vectorstore() -> Redis:
    """Create the Redis vectorstore."""

    try:
        vectorstore = Redis.from_existing_index(
            embedding=embeddings,
            index_name=INDEX_NAME,
            redis_url=REDIS_URL
        )
        return vectorstore
    except:
        pass

    # Load Redis with documents
    vectorstore = Redis.from_documents(
        documents=texts,
        embedding=embeddings,
        index_name=INDEX_NAME,
        redis_url=REDIS_URL
    )
    return vectorstore


redis = get_vectorstore()

## Specify the prompt template

PromptTemplate defines the exect text of the response that would be fed to the LLM. This step is optional, but the defaults usually work well for OpenAI and might fall short for other models.

In [40]:
def get_prompt():
    """Create the QA chain."""
    from langchain.prompts import PromptTemplate
    from langchain.chains import RetrievalQA

    # Define our prompt
    prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, say that you don't know, don't try to make up an answer.

    This should be in the following format:

    Question: [question here]
    Answer: [answer here]

    Begin!

    Context:
    ---------
    {context}
    ---------
    Question: {question}
    Answer:"""

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    return prompt

### Putting it all together

This is where the Langchain brings all the components together in a form of a simple QnA chain

In [41]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=redis.as_retriever(),
    #return_source_documents=True,
    chain_type_kwargs={"prompt": get_prompt()},
    #verbose=True
    )

### Debugging Redis

The code block below is example of how you can interact with the Redis Database

In [42]:
#!redis-cli $REDIS_CONN keys "*"
#!redis-cli $REDIS_CONN HGETALL "doc:qna:idx:063955c855a7436fbf9829821332ed2a"

###-- FLUSHDB will wipe out the entire database!!! Use with caution --###
#!redis-cli $REDIS_CONN flushdb


## Finally - let's ask questions!

Examples:
- What did the president say about Kentaji Brown Jackson
- Did he mention Stephen Breyer?
- What was his stance on Ukraine

In [43]:
query = "What did the president say about Kentaji Brown Jackson?"
res=qa(query)
res['result']

" He said she was one of the nation's top legal minds."